In [ ]:
@nbinclude("nnode.ipynb")
@nbinclude("trait.ipynb")

In [ ]:
genome_id::Int32

In [ ]:
function Genome(id, iFile)
    delimiters = " \n"
    done = 0
    #int pause;
    genome_id = id
    
    lines = readlines(iFile)
    close(iFile)
    println("\treading file")
    
    for line in lines
        println(line)
        words = split(line, [' '])
        linetype = words[1]
        list = words[2:]
        
        if linetype == "genomeend"                              # end of Genome
            idcheck = parse(Int32, list[1])
            if idcheck != genome_id
                println("ERROR: id mismatch in genome")
            end
            break
        elseif linetype == "genomestart"
            continue
        elseif linetype == "comment"                            # Comment
            continue                                            # Comment this to print the comments
            for word in list
                println(list + " ")
            end
        elseif linetype == "trait"                              # Read in a trait
            newtrait = new Trait(list)                          # Allocate the new trait
            traits.push!(newtrait)                              # Add trait to vector of traits
        elseif linetype == "node"                               # Read in a node
            newnode = NNode(list, traits)
            nodes.push!(newnode)                                # Add the node to the list of nodes
        elseif linetype == "gene"                               # Read in a Gene
            newgene = Gene(argline, traits, nodes)              # Allocate the new Gene
            genes.push!(newgene)                                # Add the gene to the genome
            println("Added gene " + newgene)
        end
    end
    println("\tfile read")
}

In [ ]:
function Genome(num_in, num_out, num_hidden, type) 
    #Temporary lists of nodes
    bias = NNode                                                       #Remember the bias

    #Assign the id 0
genome_id=0;

//Create a dummy trait (this is for future expansion of the system)
newtrait=new Trait(1,0,0,0,0,0,0,0,0,0);
traits.push_back(newtrait);

//Adjust hidden number
if (type==0) 
    num_hidden=0;
else if (type==1)
    num_hidden=num_in*num_out;

//Create the inputs and outputs

//Build the input nodes
for(ncount=1;ncount<=num_in;ncount++) {
    if (ncount<num_in)
        newnode=new NNode(SENSOR,ncount,INPUT);
    else { 
        newnode=new NNode(SENSOR,ncount,BIAS);
        bias=newnode;
    }

    //newnode->nodetrait=newtrait;

    //Add the node to the list of nodes
    nodes.push_back(newnode);
    inputs.push_back(newnode);
}

//Build the hidden nodes
for(ncount=num_in+1;ncount<=num_in+num_hidden;ncount++) {
    newnode=new NNode(NEURON,ncount,HIDDEN);
    //newnode->nodetrait=newtrait;
    //Add the node to the list of nodes
    nodes.push_back(newnode);
    hidden.push_back(newnode);
}

//Build the output nodes
for(ncount=num_in+num_hidden+1;ncount<=num_in+num_hidden+num_out;ncount++) {
    newnode=new NNode(NEURON,ncount,OUTPUT);
    //newnode->nodetrait=newtrait;
    //Add the node to the list of nodes
    nodes.push_back(newnode);
    outputs.push_back(newnode);
}

//Create the links depending on the type
if (type==0) {
    //Just connect inputs straight to outputs

    count=1;

    //Loop over the outputs
    for(curnode1=outputs.begin();curnode1!=outputs.end();++curnode1) {
        //Loop over the inputs
        for(curnode2=inputs.begin();curnode2!=inputs.end();++curnode2) {
            //Connect each input to each output
            newgene=new Gene(newtrait,0, (*curnode2), (*curnode1),false,count,0);

            //Add the gene to the genome
            genes.push_back(newgene);	 

            count++;

        }

    }

} //end type 0
//A split link from each input to each output
else if (type==1) {
    count=1; //Start the gene number counter

    curnode3=hidden.begin(); //One hidden for ever input-output pair
    //Loop over the outputs
    for(curnode1=outputs.begin();curnode1!=outputs.end();++curnode1) {
        //Loop over the inputs
        for(curnode2=inputs.begin();curnode2!=inputs.end();++curnode2) {

            //Connect Input to hidden
            newgene=new Gene(newtrait,0, (*curnode2), (*curnode3),false,count,0);
            //Add the gene to the genome
            genes.push_back(newgene);

            count++; //Next gene

            //Connect hidden to output
            newgene=new Gene(newtrait,0, (*curnode3), (*curnode1),false,count,0);
            //Add the gene to the genome
            genes.push_back(newgene);

            ++curnode3; //Next hidden node
            count++; //Next gene

        }
    }

}//end type 1
//Fully connected 
else if (type==2) {
    count=1; //Start gene counter at 1


    //Connect all inputs to all hidden nodes
    for(curnode1=hidden.begin();curnode1!=hidden.end();++curnode1) {
        //Loop over the inputs
        for(curnode2=inputs.begin();curnode2!=inputs.end();++curnode2) {
            //Connect each input to each hidden
            newgene=new Gene(newtrait,0, (*curnode2), (*curnode1),false,count,0);

            //Add the gene to the genome
            genes.push_back(newgene);	 

            count++;

        }
    }

    //Connect all hidden units to all outputs
    for(curnode1=outputs.begin();curnode1!=outputs.end();++curnode1) {
        //Loop over the inputs
        for(curnode2=hidden.begin();curnode2!=hidden.end();++curnode2) {
            //Connect each input to each hidden
            newgene=new Gene(newtrait,0, (*curnode2), (*curnode1),false,count,0);

            //Add the gene to the genome
            genes.push_back(newgene);	 

            count++;

        }
    }

    //Connect the bias to all outputs
    for(curnode1=outputs.begin();curnode1!=outputs.end();++curnode1) {
        newgene=new Gene(newtrait,0, bias, (*curnode1),false,count,0);

        //Add the gene to the genome
        genes.push_back(newgene);	 

        count++;
    }

    //Recurrently connect the hidden nodes
    for(curnode1=hidden.begin();curnode1!=hidden.end();++curnode1) {
        //Loop Over all Hidden
        for(curnode2=hidden.begin();curnode2!=hidden.end();++curnode2) {
            //Connect each hidden to each hidden
            newgene=new Gene(newtrait,0, (*curnode2), (*curnode1),true,count,0);

            //Add the gene to the genome
            genes.push_back(newgene);	 

            count++;

        }

    }

}//end type 2

                                                                                                                                            end